In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200629']:
    print('----------------------------------------------------------------')
    print(y)
    
#     exe_path = 'G:\\7z1900-extra\\7za.exe'
#     rar_path = 'L:\\KR\\data\\quant360_data\\2020\\202006\\' + y + '\\SH\\snapshot.7z'
#     path = 'A:\\KR_daily_data\\' +  y
#     os.mkdir(path)
#     path1 = path + '\\SH'
#     un_path = path1
#     cmd = '{} x {} -o{} -aos -r'.format(exe_path, rar_path, un_path)
#     os.system(cmd)
#     rar_path = 'L:\\KR\\data\\quant360_data\\2020\\202006\\' + y + '\\SH\\tick.7z'
#     cmd = '{} x {} -o{} -aos -r'.format(exe_path, rar_path, un_path)
#     os.system(cmd)
#     rar_path = 'L:\\KR\\data\\quant360_data\\2020\\202006\\' + y + '\\SZ\\snapshot.7z'
#     path1 = path + '\\SZ'
#     un_path = path1
#     cmd = '{} x {} -o{} -aos -r'.format(exe_path, rar_path, un_path)
#     os.system(cmd)
#     rar_path = 'L:\\KR\\data\\quant360_data\\2020\\202006\\' + y + '\\SZ\\order.7z'
#     cmd = '{} x {} -o{} -aos -r'.format(exe_path, rar_path, un_path)
#     os.system(cmd)
#     rar_path = 'L:\\KR\\data\\quant360_data\\2020\\202006\\' + y + '\\SZ\\tick.7z'
#     cmd = '{} x {} -o{} -aos -r'.format(exe_path, rar_path, un_path)
#     os.system(cmd)
    
    
    
    
    readPath = 'A:\\KR_daily_data\\' + y + '\\SH\\snapshot\\Level2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
    logSH1 = []
    ll = []
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        logSH1 += [df]
    del df
    logSH1 = pd.concat(logSH1).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    
    for i in range(1, 6):
        if i == 1:
            logSH1["bid" + str(i) + "p"] = logSH1["BidPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSH1["ask" + str(i) + "p"] = logSH1["OfferPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSH1["bid" + str(i) + "q"] = logSH1["BidOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
            logSH1["ask" + str(i) + "q"] = logSH1["OfferOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
        else:
            logSH1["bid" + str(i) + "p"] = logSH1["BidPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSH1["ask" + str(i) + "p"] = logSH1["OfferPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSH1["bid" + str(i) + "q"] = logSH1["BidOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
            logSH1["ask" + str(i) + "q"] = logSH1["OfferOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
    logSH1 = logSH1.rename(columns={"Volume":"cum_volume", "Amount":"cum_amount", "LastPx":"close", "OpenPx":"openPrice",
                                   "NumTrades":"numTrades"})
    logSH1["time"] = (logSH1["QuotTime"] - int(y)*1000000000).astype(np.int64)
        
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SH_*'))[0]
    path2 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdLog_SZ_*'))[0]
    path3 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zt_96_04_day_96data\\mdLog_SH_*'))[0]
    
    
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(path3,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    print(datetime.datetime.now() - startTm)
    logSH2["time"] = logSH2["time"].apply(lambda x: int((x.replace(':', "")).replace(".", ""))) 
    display(len(ll))
    display(len(set(logSH2["StockID"].unique()) & set(ll)))
    
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000) \
              & (logSH2["source"] == 13)]
    data1["cum_amount"] = data1["cum_amount"].round(2)
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "time", "numTrades"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()) - set(ll)))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()) - set(ll))
    print(len(set(data2_1["StockID"].unique()) - set(data1_1["StockID"].unique())))
    print(set(data2_1["StockID"].unique()) - set(data1_1["StockID"].unique()))
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["source"].count()
    n2 = test["IOPV"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["IOPV"])])
        display(len(test[np.isnan(test["IOPV"])])/n1)
        display(len(test[np.isnan(test["IOPV"])]["time"].unique()))
        display(test[np.isnan(test["IOPV"])]["time"].unique())
        display(len(test[np.isnan(test["IOPV"])]["StockID"].unique()))
        display(test[np.isnan(test["IOPV"])]["StockID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["source"])])
        display(n2-n1)
        print((n2-n1)/n1)
    del logSH2
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    print('----------------------------------------------------------------')
    print('SH index data:')

    readPath = 'A:\\KR_daily_data\\' + y + '\\SH\\snapshot\\Level2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs == 16) | (dateLs == 300) | (dateLs == 852) | (dateLs == 905)]
    index = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        df = pd.read_csv(i)
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        index += [df]
    index = pd.concat(index).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    
    for i in range(1, 6):
        if i == 1:
            index["bid" + str(i) + "p"] = index["BidPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            index["ask" + str(i) + "p"] = index["OfferPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            index["bid" + str(i) + "q"] = index["BidOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
            index["ask" + str(i) + "q"] = index["OfferOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
        else:
            index["bid" + str(i) + "p"] = index["BidPrice"].apply(lambda x: float(x.split(',')[i-1]))
            index["ask" + str(i) + "p"] = index["OfferPrice"].apply(lambda x: float(x.split(',')[i-1]))
            index["bid" + str(i) + "q"] = index["BidOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
            index["ask" + str(i) + "q"] = index["OfferOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
    index = index.rename(columns={"Volume":"cum_volume", "Amount":"cum_amount", "LastPx":"close", "OpenPx":"openPrice",
                                   "NumTrades":"numTrades"})
    index["time"] = (index["SendingTime"] - int(y)*1000000000).astype(np.int64)
#     index["time1"] = (index["SendingTime"] - int(y)*1000000000).astype(np.int64)
    
    startTm = datetime.datetime.now()
    logSH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice"]]
    print(datetime.datetime.now() - startTm)
    logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    
    print(index["StockID"].unique())
    
    in_dex = [16, 300, 852, 905]
    data1 = logSH[(logSH["StockID"].isin(in_dex)) & (logSH["time"] >= 91500000) & (logSH["time"] <= 150000000)]
    data2 = index[(index["StockID"].isin(in_dex)) & (index["time"] >= 91500000) & (index["time"] <= 150000000)]

    data2["close"] = data2["close"].round(4)
    data1["close"] = data1["close"].round(4)
    data2["openPrice"] = data2["openPrice"].round(4)
    data1["openPrice"] = data1["openPrice"].round(4)
    data2["cum_amount"] = data2["cum_amount"].round(1)
    data1["cum_amount"] = data1["cum_amount"].round(1)

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
    
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["source"].count()
    n2 = test["IOPV"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["IOPV"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["source"])])
    
    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice", "time"]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["source"].count()
    n2 = test["IOPV"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["IOPV"])])
    if (n2 == len1) & (n1 < len1):
        display("baseline is not complete::")
        display(test[np.isnan(test["source"])])
    
    del index
    del logSH
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    
    print('----------------------------------------------------------------')
    print('SZ lv2 data:')
    
    readPath = 'A:\\KR_daily_data\\' + y + '\\SZ\\snapshot\\Level2\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    logSZ1 = []
    ll = []
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["StockID"] = int(os.path.basename(i).split('.')[0])
        logSZ1 += [df]
    del df
    logSZ1 = pd.concat(logSZ1).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    for i in range(1, 6):
        if i == 1:
            logSZ1["bid" + str(i) + "p"] = logSZ1["BidPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSZ1["ask" + str(i) + "p"] = logSZ1["OfferPrice"].apply(lambda x: float(x.split(',')[0][1:]))
            logSZ1["bid" + str(i) + "q"] = logSZ1["BidOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
            logSZ1["ask" + str(i) + "q"] = logSZ1["OfferOrderQty"].apply(lambda x: int(x.split(',')[0][1:]))
        else:
            logSZ1["bid" + str(i) + "p"] = logSZ1["BidPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSZ1["ask" + str(i) + "p"] = logSZ1["OfferPrice"].apply(lambda x: float(x.split(',')[i-1]))
            logSZ1["bid" + str(i) + "q"] = logSZ1["BidOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
            logSZ1["ask" + str(i) + "q"] = logSZ1["OfferOrderQty"].apply(lambda x: int(x.split(',')[i-1]))
    logSZ1 = logSZ1.rename(columns={"Volume":"cum_volume", "Amount":"cum_amount", "LastPx":"close", "OpenPx":"openPrice",
                                   "NumTrades":"numTrades"})
    logSZ1["time"] = (logSZ1["QuotTime"] - int(y)*1000000000).astype(np.int64)
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    logSZ = pd.read_csv(path2,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                              "exchange", "time", "cum_volume", "cum_amount", "close",
                                              "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                              "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                              "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                              "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSZ["time"] = logSZ["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
    display(len(ll))
    display(len(set(logSZ["StockID"].unique()) & set(ll)))
    print(datetime.datetime.now() - startTm)
    
    startTm = datetime.datetime.now()
    data1 = logSZ[(logSZ["time"] >= 91500000) & (logSZ["time"] < 150000000) & (logSZ["source"] == 4)]
    data2 = logSZ1[(logSZ1["time"] >= 91500000) & (logSZ1["time"] < 150000000)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
#     data1_1["cum_amount"] = (data1_1["cum_amount"]*1000).round(0)
#     data1_1["bid1p"] = (data1_1["bid1p"]*1000).round(0)
#     data1_1["bid2p"] = (data1_1["bid2p"]*1000).round(0)
#     data1_1["bid3p"] = (data1_1["bid3p"]*1000).round(0)
#     data1_1["bid4p"] = (data1_1["bid4p"]*1000).round(0)
#     data1_1["bid5p"] = (data1_1["bid5p"]*1000).round(0)
#     data1_1["ask1p"] = (data1_1["ask1p"]*1000).round(0)
#     data1_1["ask2p"] = (data1_1["ask2p"]*1000).round(0)
#     data1_1["ask3p"] = (data1_1["ask3p"]*1000).round(0)
#     data1_1["ask4p"] = (data1_1["ask4p"]*1000).round(0)
#     data1_1["ask5p"] = (data1_1["ask5p"]*1000).round(0)
#     data1_1["close"] = (data1_1["close"]*1000).round(0)
#     data1_1["openPrice"] = (data1_1["openPrice"]*1000).round(0)
    data1_1["cum_amount"] = data1_1["cum_amount"].round(2)
    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()) - set(ll)))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()) - set(ll))
    print(len(set(data2_1["StockID"].unique()) - set(data1_1["StockID"].unique())))
    print(set(data2_1["StockID"].unique()) - set(data1_1["StockID"].unique()))
    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    print(datetime.datetime.now() - startTm)
    n1 = test["source"].count()
    n2 = test["ImageStatus"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(test[np.isnan(test["ImageStatus"])])
        print(len(test[np.isnan(test["ImageStatus"])])/n1)
        print(np.sort(test[np.isnan(test["ImageStatus"])]["time"].unique()))
        print(len(np.sort(test[np.isnan(test["ImageStatus"])]["StockID"].unique())))
        print(np.sort(test[np.isnan(test["ImageStatus"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(test[np.isnan(test["source"])])
        display(n2-n1)
    del logSZ
    del logSZ1
    del data1
    del data2
    del test
    del data1_1
    del data2_1
    
    
    
    
    
    readPath = 'A:\\KR_daily_data\\' + y + '\\SZ\\order\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    OrderLogSZ1 = []
    ll = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i, encoding='GBK')
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        OrderLogSZ1 += [df]
    OrderLogSZ1 = pd.concat(OrderLogSZ1).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    OrderLogSZ1 = OrderLogSZ1.rename(columns={"OrdType": "OrderType"})
    OrderLogSZ1["Price"] = OrderLogSZ1["Price"]*10000
    OrderLogSZ1["Price"] = OrderLogSZ1["Price"].round(0)
    OrderLogSZ1["TransactTime"] = (OrderLogSZ1["TransactTime"] - int(y) * 1000000000).astype(np.int64)
#     OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["Side"] != 'F']
    display(OrderLogSZ1["Side"].unique())
    display(OrderLogSZ1["ChannelNo"].unique())
    OrderLogSZ1["Side"] = np.where(OrderLogSZ1["Side"] == '1', 1, np.where(
    OrderLogSZ1["Side"] == '2', 2, OrderLogSZ1["Side"]))
    display(OrderLogSZ1[((OrderLogSZ1["Side"] != 1) & (OrderLogSZ1["Side"] != 2)) | (OrderLogSZ1["OrderType"].isnull())])


    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdOrderLog_*'))[0]
    OrderLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "Side", "OrderType", "Price",
                                                 "OrderQty"]]
    OrderLogSZ["OrderType"] = OrderLogSZ["OrderType"].apply(lambda x: str(x))
    
    OrderLogSZ1["OrderType"] = np.where(OrderLogSZ1["OrderType"] == 2, '2', np.where(
        OrderLogSZ1["OrderType"] == 1, '1', OrderLogSZ1['OrderType']))
    
    display(len(ll))
    display(len(set(OrderLogSZ["SecurityID"].unique()) & set(ll)))
    display(len(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()) - set(ll)))
    display(len(OrderLogSZ["SecurityID"].unique()))
    display(len(OrderLogSZ1["SecurityID"].unique()))
    display(set(OrderLogSZ["SecurityID"].unique()) - set(OrderLogSZ1["SecurityID"].unique()) - set(ll))
    display(set(OrderLogSZ1["SecurityID"].unique()) - set(OrderLogSZ["SecurityID"].unique()))


    sl = list(set(OrderLogSZ["SecurityID"].unique()) & set(OrderLogSZ1['SecurityID'].unique()))
    OrderLogSZ = OrderLogSZ[OrderLogSZ["SecurityID"].isin(sl)]
    OrderLogSZ1 = OrderLogSZ1[OrderLogSZ1["SecurityID"].isin(sl)]
    print(len(OrderLogSZ["SecurityID"].unique()))
    print(len(OrderLogSZ1["SecurityID"].unique()))
    
    print('----------------------------------------------------------------')
    print('SZ order data:')
    
    columns = ["ApplSeqNum", "TransactTime","Side",'OrderType', 'Price', 'OrderQty', "SecurityID"]
    ree = pd.merge(OrderLogSZ, OrderLogSZ1, on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo"].count()
    n2 = ree["SendingTime"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(ree[np.isnan(ree["SendingTime"])])
        print(len(ree[np.isnan(ree["SendingTime"])]))
        print(np.sort(ree[np.isnan(ree["SendingTime"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["SendingTime"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["SendingTime"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("test is complete, baseline is not complete:")
        display(ree[np.isnan(ree["sequenceNo"]) & (ree["Side"] != 'F')])
        display(ree[np.isnan(ree["sequenceNo"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo"]) & (ree["Side"] != 'F')]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo"]) & (ree["Side"] != 'F')]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo"]) & (ree["Side"] != 'F')]["SecurityID"].unique())
        display(n2-n1)
    del OrderLogSZ
    del OrderLogSZ1
    del ree
    
    
    
    
    
    readPath = 'A:\\KR_daily_data\\' + y + '\\SH\\tick\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
    SH1 = []
    ll = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        SH1 += [df]
    SH1 = pd.concat(SH1).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    SH1["TransactTime"] = (SH1["TradeTime"] - int(y) * 1000000000).astype(np.int64)
    SH1["TradePrice"] = SH1["TradePrice"] * 10000
    SH1["TradePrice"] = SH1["TradePrice"].round(0)
    SH1["TradeMoney"] = SH1["TradeAmount"] * 10000
    SH1["TradeMoney"] = SH1["TradeMoney"].round(0)
    SH1["ExecType"] = 'F'
    SH1 = SH1.rename(columns={"TradeIndex":"ApplSeqNum", "BuyNo":"BidApplSeqNum", "SellNo":"OfferApplSeqNum"})

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]
    SH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    SH = SH[SH["exchId"] == 1]
    
    display(len(ll))
    display(len(set(SH["SecurityID"].unique()) & set(ll)))
    display(len(SH["SecurityID"].unique()))
    display(len(SH1["SecurityID"].unique()))
    display(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()) - set(ll)))
    display(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()) - set(ll))
    display(len(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique())))
    display(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
    
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)
    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo"].count()
    n2 = re["TradeTime"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["TradeTime"])])
        print(len(re[np.isnan(re["TradeTime"])]))
        print(np.sort(re[np.isnan(re["TradeTime"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["TradeTime"])]["SecurityID"].unique()))
        print(re[np.isnan(re["TradeTime"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
        display(n2-n1)
    del SH
    del SH1
    del re
    
    
    
    
    
    
    readPath = 'A:\\KR_daily_data\\' + y + '\\SZ\\tick\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    TradeLogSZ1 = []
    ll = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        TradeLogSZ1 += [df]
    TradeLogSZ1 = pd.concat(TradeLogSZ1).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    TradeLogSZ1["TransactTime"] = (TradeLogSZ1["TransactTime"] - int(y) * 1000000000).astype(np.int64)
    TradeLogSZ1["TradePrice"] = TradeLogSZ1["Price"] * 10000
    TradeLogSZ1["TradePrice"] = TradeLogSZ1["TradePrice"].round(0)
    TradeLogSZ1 = TradeLogSZ1.rename(columns={"Qty":"TradeQty"})
    TradeLogSZ1["TradeMoney"] = (TradeLogSZ1["TradePrice"] * TradeLogSZ1["TradeQty"]).astype(np.int64)
    TradeLogSZ1["TradeBSFlag"] = 'N'
    
    startTm = datetime.datetime.now()
    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]

    TradeLogSZ = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    TradeLogSZ = TradeLogSZ[TradeLogSZ["exchId"] != 1]
    print(datetime.datetime.now() - startTm)
    
    
    display(len(ll))
    display(len(set(TradeLogSZ["SecurityID"].unique()) & set(ll)))
    display(len(TradeLogSZ["SecurityID"].unique()))
    display(len(TradeLogSZ1["SecurityID"].unique()))
    display(len(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()) - set(ll)))
    display(set(TradeLogSZ["SecurityID"].unique()) - set(TradeLogSZ1["SecurityID"].unique()) - set(ll))
    display(len(set(TradeLogSZ1["SecurityID"].unique()) - set(TradeLogSZ["SecurityID"].unique())))
    display(set(TradeLogSZ1["SecurityID"].unique()) - set(TradeLogSZ["SecurityID"].unique()))

    sl = list(set(TradeLogSZ["SecurityID"].unique()) & set(TradeLogSZ1['SecurityID'].unique()))
    TradeLogSZ = TradeLogSZ[TradeLogSZ["SecurityID"].isin(sl)]
    TradeLogSZ1 = TradeLogSZ1[TradeLogSZ1["SecurityID"].isin(sl)]
    print(len(TradeLogSZ["SecurityID"].unique()))
    print(len(TradeLogSZ1["SecurityID"].unique()))

    print(TradeLogSZ1.columns)
    
    
    print('----------------------------------------------------------------')
    print('SZ trade data:')
    
    TradeLogSZ["ExecType"] = TradeLogSZ["ExecType"].apply(lambda x: str(x))
    TradeLogSZ1["ExecType"] = TradeLogSZ1["ExecType"].apply(lambda x: str(x))

    columns = ["TransactTime","ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag","TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum","OfferApplSeqNum"]
    re = pd.merge(TradeLogSZ, TradeLogSZ1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo"].count()
    n2 = re["Price"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["Price"])])
        print(len(re[np.isnan(re["Price"])]))
        print(np.sort(re[np.isnan(re["Price"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["Price"])]["SecurityID"].unique()))
        print(re[np.isnan(re["Price"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
        display(n2-n1)
    del TradeLogSZ
    del TradeLogSZ1
    del re

----------------------------------------------------------------
20200629
0:03:27.806735
0:00:58.291933


0

0

----------------------------------------------------------------
SH lv2 data:


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

0

set()

8

{2989, 300028, 300104, 300824, 300837, 300838, 300841, 300842}

2209
2209
Index(['ApplSeqNum', 'BidApplSeqNum', 'SendingTime', 'Price', 'ChannelNo',
       'TradeQty', 'OfferApplSeqNum', 'Amt', 'ExecType', 'TransactTime',
       'SecurityID', 'TradePrice', 'TradeMoney', 'TradeBSFlag'],
      dtype='object')
----------------------------------------------------------------
SZ trade data:
32986579
32986579
32986579
-----------------------------------------------


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)



for y in ['20200622']:
    exe_path = 'G:\\7z1900-extra\\7za.exe'
    rar_path = 'L:\\KR\\data\\quant360_data\\2020\\202006\\' + y + '\\SH\\snapshot.7z'
    path = 'A:\\KR_daily_data\\' +  y
    os.mkdir(path)
    path1 = path + '\\SH'
    un_path = path1
    cmd = '{} x {} -o{} -aos -r'.format(exe_path, rar_path, un_path)
    os.system(cmd)

In [36]:
data1[(data1["StockID"] == 16) & (data1["cum_volume"] >= 8916488) & (data1["openPrice"] != 0)][["StockID", "time", "cum_volume", "cum_amount", "close", "openPrice"]].head()

,StockID,time,cum_volume,cum_amount,close,openPrice
3282453,16,102149000,8916488,1.672281e+10,2876.7398,2879.8483
3286512,16,102153000,8922953,1.673639e+10,2877.0522,2879.8483
3291128,16,102159000,8927214,1.674596e+10,2877.1843,2879.8483
3295157,16,102204000,8934035,1.675558e+10,2877.1244,2879.8483
3299304,16,102209000,8940584,1.677038e+10,2877.0853,2879.8483


In [37]:
data2[(data2["StockID"] == 300) & (data2["cum_volume"] >= 8916488)& (data2["openPrice"] != 0)][["StockID", "time", "cum_volume", "cum_amount", "close", "openPrice"]].head()

,StockID,time,cum_volume,cum_amount,close,openPrice
3987,300,93539000,8927740,1.499477e+10,4003.6775,4018.6094
3988,300,93544000,8993820,1.510880e+10,4003.8332,4018.6094
3989,300,93549000,9087015,1.526660e+10,4004.4771,4018.6094
3990,300,93554000,9187382,1.542987e+10,4004.2736,4018.6094
3991,300,93559000,9414975,1.563606e+10,4005.0305,4018.6094


In [4]:
logSH.head()

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
0,1592354220028853,1,3,601388,SH,83656700,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
1,1592354220028896,2,3,600331,SH,83656700,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
2,1592354220028901,3,3,600795,SH,83656700,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
3,1592354220028903,4,3,600093,SH,83656700,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
4,1592354220028909,5,3,600052,SH,83656700,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0


In [5]:
index.head()

,numTrades,OfferTradeMaxDuration,ImageStatus,TotalBidNumber,TotalWarrantExecQty,WithdrawSellMoney,IOPV,BidOrders,ETFSellAmount,TotalOfferQty,WithdrawBuyNumber,WeightedAvgOfferPx,ETFBuyNumber,WarLowerPx,MsgSeqNum,WithdrawSellAmount,ETFSellMoney,cum_volume,BidOrderQty,openPrice,HighPx,PreClosePx,LowPx,WeightedAvgBidPx,ETFSellNumber,OfferNumOrders,WithdrawSellNumber,ETFBuyAmount,TotalOfferNumber,OfferPrice,NumOfferOrders,BidPrice,OfferOrderQty,TotalBidQty,SendingTime,ETFBuyMoney,InstrumentStatus,WithdrawBuyAmount,ClosePx,BidTradeMaxDuration,NumBidOrders,close,cum_amount,AveragePx,WarUpperPx,YieldToMaturity,BidNumOrders,WithdrawBuyMoney,TradingPhaseCode,QuotTime,OfferOrders,StockID,bid1p,ask1p,bid1q,ask1q,bid2p,ask2p,bid2q,ask2q,bid3p,ask3p,bid3q,ask3q,bid4p,ask4p,bid4q,ask4q,bid5p,ask5p,bid5q,ask5q,time
0,0,0,1,0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",0,0,0,0.0,0,0.0,5611,0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0.0,0.0,2878.9924,0.0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0,0,0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]",0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]","[0,0,0,0,0,0,0,0,0,0]",0,20200617084524000,0.0,NaN,0,0.0,0,0,2878.9722,0.0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0]",0.0,NaN,20200617084454000,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",16,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,84454000
1,0,0,1,0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",0,0,0,0.0,0,0.0,5777,0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0.0,0.0,2878.9924,0.0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0,0,0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]",0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]","[0,0,0,0,0,0,0,0,0,0]",0,20200617084554000,0.0,NaN,0,0.0,0,0,2878.9722,0.0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0]",0.0,NaN,20200617084551000,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",16,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,84551000
2,0,0,1,0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",0,0,0,0.0,0,0.0,5812,0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0.0,0.0,2878.9924,0.0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0,0,0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]",0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]","[0,0,0,0,0,0,0,0,0,0]",0,20200617084559000,0.0,NaN,0,0.0,0,0,2878.9722,0.0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0]",0.0,NaN,20200617084554000,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",16,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,84554000
3,0,0,1,0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",0,0,0,0.0,0,0.0,5969,0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0.0,0.0,2878.9924,0.0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0,0,0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]",0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]","[0,0,0,0,0,0,0,0,0,0]",0,20200617084624000,0.0,NaN,0,0.0,0,0,2878.9722,0.0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0]",0.0,NaN,20200617084554000,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",16,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,84554000
4,0,0,1,0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",0,0,0,0.0,0,0.0,6135,0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0.0,0.0,2878.9924,0.0,0.0,0,"[0,0,0,0,0,0,0,0,0,0]",0,0,0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]",0,"[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]","[0,0,0,0,0,0,0,0,0,0]",0,20200617084654000,0.0,NaN,0,0.0,0,0,2878.9722,0.0,0.0,0.0,0.0,"[0,0,0,0,0,0,0,0,0,0]",0.0,NaN,20200617084651000,"[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0...",16,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0,0,0,84651000


In [13]:
logSH2[(logSH2["StockID"] == 601969) & (logSH2["source"] == 13)][["clockAtArrival", "sequenceNo", "source", "StockID", "exchange", "time", "close"]]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,close
7205,1591577129995400,24982,13,601969,SH,84528000,0.0
10522,1591577190002968,32743,13,601969,SH,84629000,0.0
13751,1591577250003221,40416,13,601969,SH,84729000,0.0
16964,1591577309997310,48073,13,601969,SH,84829000,0.0
20211,1591577369996617,55764,13,601969,SH,84929000,0.0
...,...,...,...,...,...,...,...
11661611,1591599824241123,124189868,13,601969,SH,150344000,0.0
11665902,1591599884250550,124224073,13,601969,SH,150444000,0.0
11670228,1591599944253162,124255193,13,601969,SH,150544000,0.0
11674529,1591600004243362,124283104,13,601969,SH,150644000,0.0


In [21]:
for i in logSH2.columns:
    print(i)
    print(logSH2[(logSH2["StockID"] == 600889) & (logSH2["source"] == 13)][i].unique())

clockAtArrival
[1591577111998248 1591577172005376 1591577231999659 1591577292005659
 1591577352005583 1591577412003163 1591577471996874 1591577531998196
 1591577592006492 1591577652010184 1591577711998417 1591577772003956
 1591577832003101 1591577891996468 1591577952001978 1591578012004509
 1591578071997611 1591578132002928 1591578191996821 1591578251998692
 1591578312000546 1591578371997172 1591578431998540 1591578491999940
 1591578552001107 1591578611995359 1591578671998420 1591578732000318
 1591578791997877 1591578851999874 1591578911990655 1591578972003726
 1591579031993612 1591579091998253 1591579151994125 1591579211989544
 1591579272001698 1591579331999318 1591579391992872 1591579451990947
 1591579512018046 1591579571995319 1591579631994079 1591579691994811
 1591579751994357 1591579812030652 1591579872077064 1591579932045163
 1591579991997587 1591580052012752 1591580112010850 1591580172001033
 1591580231995010 1591580291999034 1591580351998780 1591580411999438
 1591580471993636 1

[600889]
exchange
['SH']
time
[ 84510000  84610000  84710000  84810000  84910000  85010000  85110000
  85210000  85311000  85410000  85510000  85610000  85710000  85811000
  85911000  90011000  90111000  90211000  90311000  90411000  90511000
  90611000  90711000  90810000  90911000  91011000  91111000  91211000
  91311000  91411000  91511000  91611000  91711000  91811000  91911000
  92011000  92111000  92211000  92311000  92411000  92511000  92611000
  92711000  92811000  92911000  93011000  93111000  93211000  93311000
  93411000  93511000  93611000  93711000  93811000  93911000  94011000
  94111000  94211000  94311000  94411000  94511000  94611000  94711000
  94811000  94911000  95011000  95111000  95211000  95311000  95411000
  95511000  95611000  95711000  95811000  95911000 100011000 100111000
 100211000 100311000 100411000 100511000 100611000 100711000 100811000
 100911000 101011000 101111000 101211000 101311000 101411000 101511000
 101611000 101711000 101811000 101911000 102011

In [19]:
logSH2[(logSH2["StockID"] == 600175) & (logSH2["close"] != 0)]

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
11644262,1591599604627392,123896546,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11649916,1591599664623476,124074427,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11654190,1591599724604287,124124593,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11658481,1591599784590129,124166052,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11662776,1591599844574274,124206401,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11667158,1591599904669250,124238341,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11671408,1591599964694649,124268231,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11675705,1591600024694443,124295502,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0
11679846,1591600084647998,124322380,9,600175,SH,0,0,0.0,0.41,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0


In [2]:
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo"].count()
    n2 = re["TradeTime"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["TradeTime"])])
        print(len(re[np.isnan(re["TradeTime"])]))
        print(np.sort(re[np.isnan(re["TradeTime"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["TradeTime"])]["SecurityID"].unique()))
        print(re[np.isnan(re["TradeTime"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
        display(n2-n1)

15348534
15321927
15348537
-----------------------------------------------


'test is not complete:'

,clockAtArrival,sequenceNo,exchId,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice_x,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,TradeTime,TradeChannel,SendingTime,TradeAmount,TradePrice_y
8755524,1.591241e+15,65476215.0,1.0,111909280,1575751,600573,F,B,77400.0,700,54180000,2765930,2757325,NaN,NaN,NaN,NaN,NaN
8755525,1.591241e+15,65476216.0,1.0,111909340,1575752,600834,F,S,151500.0,500,75750000,2765028,2765939,NaN,NaN,NaN,NaN,NaN
8755526,1.591241e+15,65476217.0,1.0,111909340,1575753,600801,F,S,248100.0,988,245122800,2764269,2765941,NaN,NaN,NaN,NaN,NaN
8755527,1.591241e+15,65476218.0,1.0,111909420,1575754,600027,F,B,35800.0,300,10740000,2765951,2720496,NaN,NaN,NaN,NaN,NaN
8755528,1.591241e+15,65476219.0,1.0,111909420,1575755,600027,F,B,35800.0,1000,35800000,2765951,2724481,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9270804,1.591247e+15,70636798.0,1.0,130122430,1943798,603803,F,B,109400.0,100,10940000,3322035,3301957,NaN,NaN,NaN,NaN,NaN
9270805,1.591247e+15,70636799.0,1.0,130122470,1943800,603868,F,S,451800.0,100,45180000,3321688,3322041,NaN,NaN,NaN,NaN,NaN
9270806,1.591247e+15,70636800.0,1.0,130122490,1943801,600984,F,B,175200.0,100,17520000,3322042,3300828,NaN,NaN,NaN,NaN,NaN
9270807,1.591247e+15,70636801.0,1.0,130122490,1943802,600218,F,S,90200.0,200,18040000,3321983,3322043,NaN,NaN,NaN,NaN,NaN


26610
[111909270 111909280 111909300 ... 130122490 130122520 130122540]
1387
[600573 600834 600801 ... 600117 600835 603388]


In [4]:
re[np.isnan(re["TradeTime"]) & (re["SecurityID"] == 600573)] 

,clockAtArrival,sequenceNo,exchId,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice_x,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,TradeTime,TradeChannel,SendingTime,TradeAmount,TradePrice_y
8755524,1.591241e+15,65476215.0,1.0,111909280,1575751,600573,F,B,77400.0,700,54180000,2765930,2757325,NaN,NaN,NaN,NaN,NaN
8757189,1.591241e+15,65493346.0,1.0,111912720,1576074,600573,F,S,77300.0,1600,123680000,2764234,2766534,NaN,NaN,NaN,NaN,NaN
8760041,1.591241e+15,65519190.0,1.0,111918120,1576617,600573,F,B,77300.0,10000,773000000,2767479,2766534,NaN,NaN,NaN,NaN,NaN
8762043,1.591241e+15,65538995.0,1.0,111921950,1577018,600573,F,S,77200.0,1700,131240000,2759019,2768220,NaN,NaN,NaN,NaN,NaN
8762143,1.591241e+15,65540635.0,1.0,111922220,1577033,600573,F,S,77200.0,300,23160000,2759019,2768278,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9270008,1.591247e+15,70628713.0,1.0,130121670,1677999,600573,F,S,78300.0,100,7830000,2943585,2944911,NaN,NaN,NaN,NaN,NaN
9270009,1.591247e+15,70628714.0,1.0,130121700,1678000,600573,F,S,78300.0,100,7830000,2944100,2944915,NaN,NaN,NaN,NaN,NaN
9270220,1.591247e+15,70631918.0,1.0,130121910,1678014,600573,F,B,78300.0,100,7830000,2944946,2944920,NaN,NaN,NaN,NaN,NaN
9270221,1.591247e+15,70631919.0,1.0,130121910,1678015,600573,F,B,78300.0,100,7830000,2944946,2944928,NaN,NaN,NaN,NaN,NaN


In [20]:
ys[(ys["SecurityID"] == 600573) & (ys["TradeTime"] >= 20200604111850000)][["TradeTime", "TradeIndex", "SecurityID", "TradePrice", "TradeQty", "TradeMoney","TradeBSFlag","ExecType",
           "BuyNo", "SellNo"]].head(100)

,TradeTime,TradeIndex,SecurityID,TradePrice,TradeQty,TradeMoney,TradeBSFlag,ExecType,BuyNo,SellNo
6869973,20200604111854610,1574263,600573,0.0,1500,116100000.0,B,F,2763210,2757325
6869974,20200604111922340,1577042,600573,0.0,600,46320000.0,S,F,2759019,2768290
6869975,20200604111922460,1577063,600573,0.0,300,23160000.0,S,F,2759019,2768317
6869976,20200604111922550,1577077,600573,0.0,500,38600000.0,S,F,2759019,2768335
6869977,20200604111922550,1577078,600573,0.0,100,7720000.0,S,F,2759019,2768336
...,...,...,...,...,...,...,...,...,...,...
6870068,20200604112131830,1593559,600573,0.0,200,15440000.0,S,F,2791228,2794969
6870069,20200604112131830,1593560,600573,0.0,200,15440000.0,S,F,2791622,2794969
6870070,20200604112131830,1593561,600573,0.0,300,23160000.0,S,F,2791645,2794969
6870071,20200604112131830,1593563,600573,0.0,300,23160000.0,S,F,2791645,2794972


In [21]:
SH[(SH["SecurityID"] == 600573) & (SH["TransactTime"] >= 111850000)][["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]].head(20)

,TransactTime,ApplSeqNum,SecurityID,TradePrice,TradeQty,TradeMoney,TradeBSFlag,ExecType,BidApplSeqNum,OfferApplSeqNum
28292895,111854610,1574263,600573,77400,1500,116100000,B,F,2763210,2757325
28319690,111909280,1575751,600573,77400,700,54180000,B,F,2765930,2757325
28325640,111912720,1576074,600573,77300,1600,123680000,S,F,2764234,2766534
28335450,111918120,1576617,600573,77300,10000,773000000,B,F,2767479,2766534
28342127,111921950,1577018,600573,77200,1700,131240000,S,F,2759019,2768220
28342601,111922220,1577033,600573,77200,300,23160000,S,F,2759019,2768278
28342606,111922250,1577038,600573,77200,500,38600000,S,F,2759019,2768285
28343064,111922340,1577042,600573,77200,600,46320000,S,F,2759019,2768290
28343084,111922460,1577063,600573,77200,300,23160000,S,F,2759019,2768317
28343557,111922550,1577077,600573,77200,500,38600000,S,F,2759019,2768335


In [18]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
import json
pd.set_option("max_columns", 300)

f1 = open("E:\\ms.json")
df1 = json.load(f1)
df1 = pd.DataFrame.from_dict(df1,orient='index')
df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220
1601360,20180102,20180103,20180104,20180105,20180108,20180109,20180110,20180111,20180112,20180115,20180116,20180117,20180118,20180119,20180122,20180123,20180124,20180125,20180126,20180129,20180130,20180131,20180201,20180202,20180205,20180206,20180207,20180208,20180209,20180212,20180213,20180214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001872,20180710,20180711,20180712,20180713,20180716,20180717,20180718,20180719,20180720,20180723,20180724,20180725,20180726,20180727,20180730,20180731,20180801,20180802,20180803,20180806,20180807,20180808,20180809,20180810,20180813,20180814,20180815,20180816,20180817,20180820,20180821,20180822,20180823.0,20180824.0,20180827.0,20180828.0,20180829.0,20180830.0,20180831.0,20180903.0,20180904.0,20180905.0,20180906.0,20180907.0,20180910.0,20180911.0,20180912.0,20180913.0,20180914.0,20180917.0,20180918.0,20180919.0,20180920.0,20180921.0,20180925.0,20180926.0,20180927.0,20181010.0,20181011.0,20181012.0,20181015.0,20181016.0,20181017.0,20181018.0,20181019.0,20181022.0,20181023.0,20181024.0,20181025.0,20181026.0,20181029.0,20181030.0,20181031.0,20181101.0,20181102.0,20181105.0,20181106.0,20181107.0,20181108.0,20181109.0,20181112.0,20181113.0,20181114.0,20181115.0,20181116.0,20181119.0,20181120.0,20181121.0,20181122.0,20181123.0,20181126.0,20181127.0,20181128.0,20181129.0,20181130.0,20181203.0,20181204.0,20181205.0,20181206.0,20181207.0,20181210.0,20181211.0,20181212.0,20181213.0,20181214.0,20181217.0,20181218.0,20181219.0,20181220.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001914,20190102,20190103,20190104,20190107,20190108,20190109,20190110,20190111,20190114,20190115,20190116,20190117,20190118,20190121,20190122,20190123,20190124,20190125,20190128,20190129,20190130,20190131,20190201,20190211,20190212,20190213,20190214,20190215,20190218,20190219,20190220,20190221,20190222.0,20190225.0,20190226.0,20190227.0,20190228.0,20190301.0,20190304.0,20190305.0,20190306.0,20190307.0,20190308.0,20190311.0,20190312.0,20190313.0,20190314.0,20190315.0,20190318.0,20190319.0,20190320.0,20190321.0,20190322.0,20190325.0,20190326.0,20190327.0,20190328.0,20190329.0,

In [2]:
logSZ1 = logSZ1.rename(columns={"成交总量":"cum_volume", "成交总金额":"cum_amount", "最新价":"close", "开始价":"openPrice"})
for i in range(1, 6):
    logSZ1 = logSZ1.rename(columns={"申买价" + str(i): "bid" + str(i) + "p", "申卖价" + str(i): "ask" + str(i) + "p",
                                   "申买量" + str(i): "bid" + str(i) + "q", "申卖量" + str(i): "ask" + str(i) + "q"})
logSZ1["time"] = logSZ1["日期时间"] - 20190610000000000
logSZ1 = logSZ1[["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", "openPrice", "time"]]

In [20]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
y='20190610'
pd.set_option("max_columns", 200)
readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\kuanrui\\szse_20190610_csv\\szse_' + y + '\\snap\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
logSZ1 = []
startTm = datetime.datetime.now()
for i in dataPathLs[:1]:
    df = pd.read_csv(i)
    df["StockID"] = int(os.path.basename(i).split('.')[0])
    logSZ1 += [df]
del df
logSZ1 = pd.concat(logSZ1).reset_index(drop=True)
pd.set_option("max_columns", 200)
pd.set_option("max_rows", 200)
logSZ1[logSZ1["日期时间"] > 20190610093003000].head(10)

,日期时间,行情类别,交易阶段代码,昨收价,开始价,最高价,最低价,最新价,成交笔数,成交总量,成交总金额,申买价1,申买价2,申买价3,申买价4,申买价5,申买价6,申买价7,申买价8,申买价9,申买价10,申买量1,申买量2,申买量3,申买量4,申买量5,申买量6,申买量7,申买量8,申买量9,申买量10,申买价位总委托笔数1,申买价位总委托笔数2,申买价位总委托笔数3,申买价位总委托笔数4,申买价位总委托笔数5,申买价位总委托笔数6,申买价位总委托笔数7,申买价位总委托笔数8,申买价位总委托笔数9,申买价位总委托笔数10,买一价前50笔订单1,买一价前50笔订单2,买一价前50笔订单3,买一价前50笔订单4,买一价前50笔订单5,买一价前50笔订单6,买一价前50笔订单7,买一价前50笔订单8,买一价前50笔订单9,买一价前50笔订单10,买一价前50笔订单11,买一价前50笔订单12,买一价前50笔订单13,买一价前50笔订单14,买一价前50笔订单15,买一价前50笔订单16,买一价前50笔订单17,买一价前50笔订单18,买一价前50笔订单19,买一价前50笔订单20,买一价前50笔订单21,买一价前50笔订单22,买一价前50笔订单23,买一价前50笔订单24,买一价前50笔订单25,买一价前50笔订单26,买一价前50笔订单27,买一价前50笔订单28,买一价前50笔订单29,买一价前50笔订单30,买一价前50笔订单31,买一价前50笔订单32,买一价前50笔订单33,买一价前50笔订单34,买一价前50笔订单35,买一价前50笔订单36,买一价前50笔订单37,买一价前50笔订单38,买一价前50笔订单39,买一价前50笔订单40,买一价前50笔订单41,买一价前50笔订单42,买一价前50笔订单43,买一价前50笔订单44,买一价前50笔订单45,买一价前50笔订单46,买一价前50笔订单47,买一价前50笔订单48,买一价前50笔订单49,买一价前50笔订单50,申卖价1,申卖价2,申卖价3,申卖价4,申卖价5,申卖价6,申卖价7,申卖价8,申卖价9,申卖价10,申卖量1,申卖量2,申卖量3,申卖量4,申卖量5,申卖量6,申卖量7,申卖量8,申卖量9,申卖量10,申卖价位总委托笔数1,申卖价位总委托笔数2,申卖价位总委托笔数3,申卖价位总委托笔数4,申卖价位总委托笔数5,申卖价位总委托笔数6,申卖价位总委托笔数7,申卖价位总委托笔数8,申卖价位总委托笔数9,申卖价位总委托笔数10,卖一价前50笔订单1,卖一价前50笔订单2,卖一价前50笔订单3,卖一价前50笔订单4,卖一价前50笔订单5,卖一价前50笔订单6,卖一价前50笔订单7,卖一价前50笔订单8,卖一价前50笔订单9,卖一价前50笔订单10,卖一价前50笔订单11,卖一价前50笔订单12,卖一价前50笔订单13,卖一价前50笔订单14,卖一价前50笔订单15,卖一价前50笔订单16,卖一价前50笔订单17,卖一价前50笔订单18,卖一价前50笔订单19,卖一价前50笔订单20,卖一价前50笔订单21,卖一价前50笔订单22,卖一价前50笔订单23,卖一价前50笔订单24,卖一价前50笔订单25,卖一价前50笔订单26,卖一价前50笔订单27,卖一价前50笔订单28,卖一价前50笔订单29,卖一价前50笔订单30,卖一价前50笔订单31,卖一价前50笔订单32,卖一价前50笔订单33,卖一价前50笔订单34,卖一价前50笔订单35,卖一价前50笔订单36,卖一价前50笔订单37,卖一价前50笔订单38,卖一价前50笔订单39,卖一价前50笔订单40,卖一价前50笔订单41,卖一价前50笔订单42,卖一价前50笔订单43,卖一价前50笔订单44,卖一价前50笔订单45,卖一价前50笔订单46,卖一价前50笔订单47,卖一价前50笔订单48,卖一价前50笔订单49,卖一价前50笔订单50,升跌1,升跌2,委买加权平均价,买入总量,委卖加权平均价,卖出总量,市盈率,基金净值,基金实时参考净值,权证溢价率,涨停价,跌停价,实时成交量加权平均利率,加权平均利率涨跌BP,昨收盘成交量加权平均利率,期权持仓量,统计量指标样本个数,StockID
63,20190610093006000,10,T0,5410,5420,5480,5420,5480,116,599400,3257189000,5460,5450,5440,5430,5420,5410,5400,5380,5370,5360,300,36800,73700,150500,187900,24900,23900,20000,2000,600,1,15,22,43,36,4,5,1,1,1,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5480,5490,5500,5520,5540,5550,5560,5570,5580,5590,800,1800,16800,2100,1800,200,28400,27000,8200,11600,1,1,3,2,1,1,6,2,4,7,800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,0,5150,1181500,5750,450200,0,0,0,0,5950,4870,0,0,0,0,0,300313
64,20190610093009000,10,T0,5410,5420,5480,5420,5480,118,601200,3267053000,5480,5460,5450,5440,5430,5420,5410,5400,5380,5370,22100,16700,44500,85600,215700,204800,25100,23900,20700,2000,4,2,22,25,77,40,5,5,2,1,6600,7300,4100,4100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5490,5500,5520,5540,5550,5560,5570,5580,5590,5600,1800,16800,2100,1800,200,28400,27000,8200,12900,37100,1,3,2,1,1,6,2,4,8,3,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,70,0,5180,1325900,5750,463900,0,0,0,0,5950,4870,0,0,0,0,0,300313
65,20190610093012000,10,T0,5410,5420,5520,5420,5520,130,624700,3396273000,5490,5480,5460,5450,5440,5430,5420,5410,5400,5380,7500,1500,9100,64500,86000,230700,205600,25100,23900,20700,1,1,10,29,26,80,42,5,5,2,7500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5520,5540,5550,5560,5570,5580,5590,5600,5610,5620,800,1800,200,28400,27000,8200,13900,37100,5000,6000,1,1,1,6,2,4,9,3,2,1,800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,110,40,5190,1339300,5770,461900,0,0,0,0,5950,4870,0,0,0,0,0,300313
66,20190610093015000,10,T0,5410,5420,5520,5420,5480,136,633500,3444536000,5490,5480,5460,5450,5440,5430,5420,5410,5400,5380,16200,2700,9100,63700,86000,230800,205600,25100,23900,20700,1,8,10,27,26,80,42,5,5,2,16200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5520,5540,5550,5560,5570,5580,5590,5600,5610,5620,800,1800,200,28400,27000,8200,13900,37

In [31]:
logSZ1 = logSZ1.rename(columns={"成交总量":"cum_volume", "成交总金额":"cum_amount", "最新价":"close", "开始价":"open",
                               "昨收价":"prevClose", "成交笔数":"cum_tradesCnt", "最高价":"high", "最低价":"low", 
                               "买入总量":"totalBidQuantity", "卖出总量":"totalAskQuantity", "委买加权平均价":"vwapBid",
                               "委卖加权平均价":"vwapAsk"})
for i in range(1, 11):
    logSZ1 = logSZ1.rename(columns={"申买价" + str(i): "bid" + str(i) + "p", "申卖价" + str(i): "ask" + str(i) + "p",
                                   "申买量" + str(i): "bid" + str(i) + "q", "申卖量" + str(i): "ask" + str(i) + "q",
                                   "申买价位总委托笔数" + str(i): "bid" + str(i) + "n", "申卖价位总委托笔数" + str(i): "ask" + str(i) + "n"})
for i in range(1, 51):
    logSZ1 = logSZ1.rename(columns={"买一价前50笔订单" + str(i): "bid1Top" + str(i) + "q", 
                                    "卖一价前50笔订单" + str(i): "ask1Top" + str(i) + "q"})
logSZ1["time"] = ((logSZ1["日期时间"] - 20190610000000000)*1000).astype("int64")
logSZ1["date"] = logSZ1["日期时间"]//1000000000
logSZ1["clockAtArrival"] = logSZ1["日期时间"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
logSZ1["ID"] = logSZ1["StockID"] + 2000000
    
logSZ1 = logSZ1.fillna(0)
logSZ1 = logSZ1[(logSZ1["bid1p"] != 0) & (logSZ1["ask1p"] != 0)]
logSZ1["ordering"] = logSZ1.groupby("ID").cumcount()
logSZ1["ordering"] = logSZ1["ordering"] + 1

for columns in ["cum_tradesCnt", "cum_volume", 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q','ask7q','ask8q','ask9q',
                        'ask10q', 'bid1q','bid2q','bid3q','bid4q','bid5q','bid6q','bid7q','bid8q','bid9q','bid10q',
                    "totalBidQuantity", "totalAskQuantity", 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n']:
    logSZ1[columns] = logSZ1[columns].astype('int64')


for cols in ["prevClose", 'open', "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p',
             'bid2p','bid1p','ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'vwapBid', "vwapAsk"]:
    logSZ1[cols] = (logSZ1[cols] * 10).astype('int64') 
for cols in ["cum_amount"]:
    logSZ1[cols] = (logSZ1[cols] * 10).astype('int64')

logSZ1['datetime'] = logSZ1["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
for cols in ["totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]:
    logSZ1[cols] = 0
    
logSZ1["prevClose"] = np.where(logSZ1["time"] >= 91500000000, logSZ1.groupby("ID")["prevClose"].transform("max"), logSZ1["prevClose"]) 
logSZ1["open"] = np.where(logSZ1["cum_volume"] > 0, logSZ1.groupby("ID")["open"].transform("max"), logSZ1["open"])
assert(sum(logSZ1[logSZ1["open"] != 0].groupby("ID")["open"].nunique() != 1) == 0)
assert(sum(logSZ1[logSZ1["prevClose"] != 0].groupby("ID")["prevClose"].nunique() != 1) == 0)
assert(logSZ1[logSZ1["time"] >= 91500000000]["prevClose"].min() > 0)
assert(logSZ1[logSZ1["cum_volume"] > 0]["open"].min() > 0)
    
logSZ1 = logSZ1[["date", "time", "clockAtArrival", "datetime", "ID", "ordering", "cum_tradesCnt", "cum_volume", "cum_amount", "prevClose",
                            "open", "high", "low", "close", 'bid10p','bid9p','bid8p','bid7p','bid6p','bid5p','bid4p','bid3p','bid2p','bid1p',
                            'ask1p','ask2p','ask3p','ask4p','ask5p','ask6p','ask7p','ask8p','ask9p','ask10p', 'bid10q','bid9q','bid8q',
                             'bid7q','bid6q','bid5q','bid4q','bid3q','bid2q','bid1q', 'ask1q','ask2q','ask3q','ask4q','ask5q','ask6q',
                             'ask7q','ask8q','ask9q','ask10q', 'bid10n', 'bid9n', 'bid8n', 'bid7n', 'bid6n', 'bid5n', 'bid4n', 'bid3n', 'bid2n', 'bid1n', 
                             'ask1n', 'ask2n', 'ask3n', 'ask4n', 'ask5n', 'ask6n','ask7n', 'ask8n', 'ask9n', 'ask10n','bid1Top1q','bid1Top2q','bid1Top3q','bid1Top4q','bid1Top5q','bid1Top6q',
        'bid1Top7q','bid1Top8q','bid1Top9q','bid1Top10q','bid1Top11q','bid1Top12q','bid1Top13q','bid1Top14q','bid1Top15q','bid1Top16q','bid1Top17q','bid1Top18q',
        'bid1Top19q','bid1Top20q','bid1Top21q','bid1Top22q','bid1Top23q','bid1Top24q','bid1Top25q','bid1Top26q','bid1Top27q','bid1Top28q','bid1Top29q',
        'bid1Top30q','bid1Top31q','bid1Top32q','bid1Top33q','bid1Top34q','bid1Top35q','bid1Top36q','bid1Top37q','bid1Top38q','bid1Top39q','bid1Top40q',
        'bid1Top41q','bid1Top42q','bid1Top43q','bid1Top44q','bid1Top45q','bid1Top46q','bid1Top47q','bid1Top48q','bid1Top49q','bid1Top50q', 'ask1Top1q',
        'ask1Top2q','ask1Top3q','ask1Top4q','ask1Top5q','ask1Top6q','ask1Top7q','ask1Top8q','ask1Top9q','ask1Top10q','ask1Top11q','ask1Top12q','ask1Top13q',
        'ask1Top14q','ask1Top15q','ask1Top16q','ask1Top17q','ask1Top18q','ask1Top19q','ask1Top20q','ask1Top21q','ask1Top22q','ask1Top23q',
        'ask1Top24q','ask1Top25q','ask1Top26q','ask1Top27q','ask1Top28q','ask1Top29q','ask1Top30q','ask1Top31q','ask1Top32q','ask1Top33q',
        'ask1Top34q','ask1Top35q','ask1Top36q','ask1Top37q','ask1Top38q','ask1Top39q','ask1Top40q','ask1Top41q','ask1Top42q','ask1Top43q',
        'ask1Top44q','ask1Top45q','ask1Top46q','ask1Top47q','ask1Top48q','ask1Top49q','ask1Top50q',"totalBidQuantity", "totalAskQuantity","vwapBid", "vwapAsk",
        "totalBidOrders",'totalAskOrders','totalBidLevels', 'totalAskLevels', 'bidTradeMaxDuration', 'askTradeMaxDuration', 'cum_canceledBuyOrders', 'cum_canceledBuyVolume',
        "cum_canceledBuyAmount", "cum_canceledSellOrders", 'cum_canceledSellVolume',"cum_canceledSellAmount"]]
logSZ1[logSZ1["time"] > 93003000000].head(10)

,date,time,clockAtArrival,datetime,ID,ordering,cum_tradesCnt,cum_volume,cum_amount,prevClose,open,high,low,close,bid10p,bid9p,bid8p,bid7p,bid6p,bid5p,bid4p,bid3p,bid2p,bid1p,ask1p,ask2p,ask3p,ask4p,ask5p,ask6p,ask7p,ask8p,ask9p,ask10p,bid10q,bid9q,bid8q,bid7q,bid6q,bid5q,bid4q,bid3q,bid2q,bid1q,ask1q,ask2q,ask3q,ask4q,ask5q,ask6q,ask7q,ask8q,ask9q,ask10q,bid10n,bid9n,bid8n,bid7n,bid6n,bid5n,bid4n,bid3n,bid2n,bid1n,ask1n,ask2n,ask3n,ask4n,ask5n,ask6n,ask7n,ask8n,ask9n,ask10n,bid1Top1q,bid1Top2q,bid1Top3q,bid1Top4q,bid1Top5q,bid1Top6q,bid1Top7q,bid1Top8q,bid1Top9q,bid1Top10q,bid1Top11q,bid1Top12q,bid1Top13q,bid1Top14q,bid1Top15q,bid1Top16q,bid1Top17q,bid1Top18q,bid1Top19q,bid1Top20q,bid1Top21q,bid1Top22q,bid1Top23q,bid1Top24q,bid1Top25q,bid1Top26q,bid1Top27q,bid1Top28q,bid1Top29q,bid1Top30q,bid1Top31q,bid1Top32q,bid1Top33q,bid1Top34q,bid1Top35q,bid1Top36q,bid1Top37q,bid1Top38q,bid1Top39q,bid1Top40q,bid1Top41q,bid1Top42q,bid1Top43q,bid1Top44q,bid1Top45q,bid1Top46q,bid1Top47q,bid1Top48q,bid1Top49q,bid1Top50q,ask1Top1q,ask1Top2q,ask1Top3q,ask1Top4q,ask1Top5q,ask1Top6q,ask1Top7q,ask1Top8q,ask1Top9q,ask1Top10q,ask1Top11q,ask1Top12q,ask1Top13q,ask1Top14q,ask1Top15q,ask1Top16q,ask1Top17q,ask1Top18q,ask1Top19q,ask1Top20q,ask1Top21q,ask1Top22q,ask1Top23q,ask1Top24q,ask1Top25q,ask1Top26q,ask1Top27q,ask1Top28q,ask1Top29q,ask1Top30q,ask1Top31q,ask1Top32q,ask1Top33q,ask1Top34q,ask1Top35q,ask1Top36q,ask1Top37q,ask1Top38q,ask1Top39q,ask1Top40q,ask1Top41q,ask1Top42q,ask1Top43q,ask1Top44q,ask1Top45q,ask1Top46q,ask1Top47q,ask1Top48q,ask1Top49q,ask1Top50q,totalBidQuantity,totalAskQuantity,vwapBid,vwapAsk,totalBidOrders,totalAskOrders,totalBidLevels,totalAskLevels,bidTradeMaxDuration,askTradeMaxDuration,cum_canceledBuyOrders,cum_canceledBuyVolume,cum_canceledBuyAmount,cum_canceledSellOrders,cum_canceledSellVolume,cum_canceledSellAmount
63,20190610,93006000000,1560130206000000,2019-06-10 09:30:06,2300313,63,116,599400,32571890000,54100,54200,54800,54200,54800,53600,53700,53800,54000,54100,54200,54300,54400,54500,54600,54800,54900,55000,55200,55400,55500,55600,55700,55800,55900,600,2000,20000,23900,24900,187900,150500,73700,36800,300,800,1800,16800,2100,1800,200,28400,27000,8200,11600,1,1,1,5,4,36,43,22,15,1,1,1,3,2,1,1,6,2,4,7,300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1181500,450200,51500,57500,0,0,0,0,0,0,0,0,0,0,0,0
64,20190610,93009000000,1560130209000000,2019-06-10 09:30:09,2300313,64,118,601200,32670530000,54100,54200,54800,54200,54800,53700,53800,54000,54100,54200,54300,54400,54500,54600,54800,54900,55000,55200,55400,55500,55600,55700,55800,55900,56000,2000,20700,23900,25100,204800,215700,85600,44500,16700,22100,1800,16800,2100,1800,200,28400,27000,8200,12900,37100,1,2,5,5,40,77,25,22,2,4,1,3,2,1,1,6,2,4,8,3,6600,7300,4100,4100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1325900,463900,51800,57500,0,0,0,0,0,0,0,0,0,0,0,0
65,20190610,93012000000,1560130212000000,2019-06-10 09:30:12,2300313,65,130,624700,33962730000,54100,54200,55200,54200,55200,53800,54000,54100,54200,54300,54400,54500,54600,54800,54900,55200,55400,55500,55600,55700,55800,55900,56000,56100,56200,20700,23900,25100,205600,230700,86000,64500,9100,1500,7500,800,1800,200,28400,27000,8200,13900,37100,5000,6000,2,5,5,42,80,26,29,10,1,1,1,1,1,6,2,4,9,3,2,1,7500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1339300,461900,51900,57700,0,0,0,0,0,0,0,0,0,0,0,0
66,20190610,93015000000,1560130215000000,2019-06-10 09:30:15,2300313,66,136,633500,34445360000,54100,54200,55200,54200,54800,53800,54000,54100,54200,54300,54400,54500,54600,5

In [17]:
datetime.datetime.fromtimestamp(1560130221000000/1e6).strftime("%Y-%m-%d %H:%M:%S %f")

'2019-06-10 09:30:21 000000'

In [7]:
logSZ1[(logSZ1["StockID"] == 1) & (logSZ1["time"] > 93000000)].head(10)

,日期时间,行情类别,交易阶段代码,昨收价,openPrice,最高价,最低价,close,成交笔数,cum_volume,cum_amount,bid1p,bid2p,bid3p,bid4p,bid5p,申买价6,申买价7,申买价8,申买价9,申买价10,bid1q,bid2q,bid3q,bid4q,bid5q,申买量6,申买量7,申买量8,申买量9,申买量10,申买价位总委托笔数1,申买价位总委托笔数2,申买价位总委托笔数3,申买价位总委托笔数4,申买价位总委托笔数5,申买价位总委托笔数6,申买价位总委托笔数7,申买价位总委托笔数8,申买价位总委托笔数9,申买价位总委托笔数10,买一价前50笔订单1,买一价前50笔订单2,买一价前50笔订单3,买一价前50笔订单4,买一价前50笔订单5,买一价前50笔订单6,买一价前50笔订单7,买一价前50笔订单8,买一价前50笔订单9,买一价前50笔订单10,买一价前50笔订单11,买一价前50笔订单12,买一价前50笔订单13,买一价前50笔订单14,买一价前50笔订单15,买一价前50笔订单16,买一价前50笔订单17,买一价前50笔订单18,买一价前50笔订单19,买一价前50笔订单20,买一价前50笔订单21,买一价前50笔订单22,买一价前50笔订单23,买一价前50笔订单24,买一价前50笔订单25,买一价前50笔订单26,买一价前50笔订单27,买一价前50笔订单28,买一价前50笔订单29,买一价前50笔订单30,买一价前50笔订单31,买一价前50笔订单32,买一价前50笔订单33,买一价前50笔订单34,买一价前50笔订单35,买一价前50笔订单36,买一价前50笔订单37,买一价前50笔订单38,买一价前50笔订单39,买一价前50笔订单40,买一价前50笔订单41,买一价前50笔订单42,买一价前50笔订单43,买一价前50笔订单44,买一价前50笔订单45,买一价前50笔订单46,买一价前50笔订单47,买一价前50笔订单48,买一价前50笔订单49,买一价前50笔订单50,ask1p,ask2p,ask3p,ask4p,ask5p,申卖价6,申卖价7,申卖价8,申卖价9,申卖价10,ask1q,ask2q,ask3q,ask4q,ask5q,申卖量6,申卖量7,申卖量8,申卖量9,申卖量10,申卖价位总委托笔数1,申卖价位总委托笔数2,申卖价位总委托笔数3,申卖价位总委托笔数4,申卖价位总委托笔数5,申卖价位总委托笔数6,申卖价位总委托笔数7,申卖价位总委托笔数8,申卖价位总委托笔数9,申卖价位总委托笔数10,卖一价前50笔订单1,卖一价前50笔订单2,卖一价前50笔订单3,卖一价前50笔订单4,卖一价前50笔订单5,卖一价前50笔订单6,卖一价前50笔订单7,卖一价前50笔订单8,卖一价前50笔订单9,卖一价前50笔订单10,卖一价前50笔订单11,卖一价前50笔订单12,卖一价前50笔订单13,卖一价前50笔订单14,卖一价前50笔订单15,卖一价前50笔订单16,卖一价前50笔订单17,卖一价前50笔订单18,卖一价前50笔订单19,卖一价前50笔订单20,卖一价前50笔订单21,卖一价前50笔订单22,卖一价前50笔订单23,卖一价前50笔订单24,卖一价前50笔订单25,卖一价前50笔订单26,卖一价前50笔订单27,卖一价前50笔订单28,卖一价前50笔订单29,卖一价前50笔订单30,卖一价前50笔订单31,卖一价前50笔订单32,卖一价前50笔订单33,卖一价前50笔订单34,卖一价前50笔订单35,卖一价前50笔订单36,卖一价前50笔订单37,卖一价前50笔订单38,卖一价前50笔订单39,卖一价前50笔订单40,卖一价前50笔订单41,卖一价前50笔订单42,卖一价前50笔订单43,卖一价前50笔订单44,卖一价前50笔订单45,卖一价前50笔订单46,卖一价前50笔订单47,卖一价前50笔订单48,卖一价前50笔订单49,卖一价前50笔订单50,升跌1,升跌2,委买加权平均价,买入总量,委卖加权平均价,卖出总量,市盈率,基金净值,基金实时参考净值,权证溢价率,涨停价,跌停价,实时成交量加权平均利率,加权平均利率涨跌BP,昨收盘成交量加权平均利率,期权持仓量,统计量指标样本个数,StockID,time
5071296,20190610093003000,10,T0,11920,12010,12050,11980,12010,407,1189800,14287766440,12000,11980,11970,11960,11950,11940,11930,11920,11910,11900,9300,32600,3800,50800,65800,3900,17300,44500,20100,70100,2,8,4,16,11,4,18,18,17,21,7900,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12010,12020,12030,12040,12050,12060,12070,12080,12090,12100,8400,3800,27300,40200,141889,51300,60000,74600,289100,121600,5,4,6,14,23,22,13,32,47,51,4300,1300,200,600,2000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0,11520,2043300,12440,5301017,864,0,0,0,13110,10730,0,0,0,0,0,1,93003000
5071297,20190610093006000,10,T0,11920,12010,12050,11980,12000,445,1218500,14632297440,12000,11990,11980,11970,11960,11950,11940,11930,11920,11910,13000,100,28700,4300,53500,75300,3900,23500,44500,20100,2,1,8,5,18,14,4,19,18,17,6200,6800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12010,12020,12030,12040,12050,12060,12070,12080,12090,12100,7800,3500,27300,58400,146789,51300,59500,74600,343100,126700,3,4,6,20,29,22,12,32,50,58,4500,2100,1200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,0,11530,2067300,12430,5444317,863,0,0,0,13110,10730,0,0,0,0,0,1,93006000
5071298,20190610093009000,10,T0,11920,12010,12050,11980,12010,465,1243200,14929026440,12010,12000,11990,11980,11970,11960,11950,11940,11930,11920,29600,15100,100,25900,4300,56500,75800,3900,23500,46000,8,4,1,8,5,19,15,4,19,19,23000,100,1600,100,500,100,3300,900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12030,12040,12050,12060,12070,12080,12090,12100,12110,12120,27600,58800,146789,51300,59500,74600,343100,129400,40400,47200,4,21,29,22,12,32,50,58,9,32,2000,10000,10000,5600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0,11540,2091900,12430,5462217,864,0,0,0,13110,10730,0,0,0,0,0,1,93009000
5071299,20190610093012000,10,T0,11920,12010,12050,11980,12010,471,1245300,14954251440,12010,12000,11990,11980,11970,11960,11950,11940,11930,119

In [11]:
logSZ[(logSZ["StockID"] == 1) & (logSZ["time"] > 93000000)].head(10)

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,bid2p,bid3p,bid4p,bid5p,bid1q,bid2q,bid3q,bid4q,bid5q,ask1p,ask2p,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice
221756,1560130239052510,1664878,4,1,SZ,93003000,1189800,14287766.44,12.01,12.00,11.98,11.97,11.96,11.95,9300,32600,3800,50800,65800,12.01,12.02,12.03,12.04,12.05,8400,3800,27300,40200,141889,12.01
222966,1560130239456597,1719575,3,1,SZ,93003000,1189800,14287766.44,12.01,12.00,11.98,11.97,11.96,11.95,9300,32600,3800,50800,65800,12.01,12.02,12.03,12.04,12.05,8400,3800,27300,40200,141889,12.01
226032,1560130242069096,1887424,4,1,SZ,93006000,1218500,14632297.44,12.00,12.00,11.99,11.98,11.97,11.96,13000,100,28700,4300,53500,12.01,12.02,12.03,12.04,12.05,7800,3500,27300,58400,146789,12.01
226510,1560130242193173,1892756,3,1,SZ,93006000,1218500,14632297.44,12.00,12.00,11.99,11.98,11.97,11.96,13000,100,28700,4300,53500,12.01,12.02,12.03,12.04,12.05,7800,3500,27300,58400,146789,12.01
230273,1560130245029746,1987889,4,1,SZ,93009000,1243200,14929026.44,12.01,12.01,12.00,11.99,11.98,11.97,29600,15100,100,25900,4300,12.03,12.04,12.05,12.06,12.07,27600,58800,146789,51300,59500,12.01
230824,1560130245224025,1993775,3,1,SZ,93009000,1243200,14929026.44,12.01,12.01,12.00,11.99,11.98,11.97,29600,15100,100,25900,4300,12.03,12.04,12.05,12.06,12.07,27600,58800,146789,51300,59500,12.01
234458,1560130248059182,2057746,4,1,SZ,93012000,1245300,14954251.44,12.01,12.01,12.00,11.99,11.98,11.97,57500,14800,6800,15100,4300,12.03,12.04,12.05,12.06,12.07,27700,60800,146789,51300,59500,12.01
235284,1560130248277258,2061362,3,1,SZ,93012000,1245300,14954251.44,12.01,12.01,12.00,11.99,11.98,11.97,57500,14800,6800,15100,4300,12.03,12.04,12.05,12.06,12.07,27700,60800,146789,51300,59500,12.01
238441,1560130251065459,2102469,4,1,SZ,93015000,1280100,15372906.44,12.01,12.01,12.00,11.99,11.98,11.97,68200,4500,100,15100,4300,12.04,12.05,12.06,12.07,12.08,58000,146789,51300,59500,77600,12.01
238980,1560130251202281,2104692,3,1,SZ,93015000,1280100,15372906.44,12.01,12.01,12.00,11.99,11.98,11.97,68200,4500,100,15100,4300,12.04,12.05,12.06,12.07,12.08,58000,146789,51300,59500,77600,12.01


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime
pd.set_option("max_columns", 200)

for y in ['20200511']:
    readPath = '\\\\192.168.10.30\\Kevin_zhenyu\\temp\\kuanrui\\沪深一周样例数据\\' + y + '\\SH\\tick\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs >= 600000) & (dateLs <= 700000)]
    SH1 = []
    
    startTm = datetime.datetime.now()
    for i in dataPathLs:
        df = pd.read_csv(i)
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        SH1 += [df]
    SH1 = pd.concat(SH1).reset_index(drop=True)
    print(datetime.datetime.now() - startTm)
    SH1["TransactTime"] = (SH1["TradeTime"] - int(y) * 1000000000).astype(np.int64)
    SH1["TradePrice"] = SH1["TradePrice"] * 10000
    SH1["TradePrice"] = SH1["TradePrice"].round(0)
    SH1["TradeMoney"] = SH1["TradeAmount"] * 10000
    SH1["TradeMoney"] = SH1["TradeMoney"].round(0)
    SH1["ExecType"] = 'F'
    SH1 = SH1.rename(columns={"TradeIndex":"ApplSeqNum", "BuyNo":"BidApplSeqNum", "SellNo":"OfferApplSeqNum"})

    path1 = np.array(glob.glob('\\\\192.168.10.30\\Kevin_zhenyu\\rawData\\logs_' + y + '_zs_92_01_day_data\\mdTradeLog_*'))[0]
    SH = pd.read_csv(path1,
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "exchId", "TransactTime",
                                                 "ApplSeqNum", "SecurityID", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)

0:01:21.486356
1598
1598
Index(['TradeTime', 'TradeChannel', 'SendingTime', 'OfferApplSeqNum',
       'TradeAmount', 'TradeBSFlag', 'ApplSeqNum', 'TradePrice', 'TradeQty',
       'BidApplSeqNum', 'SecurityID', 'TransactTime', 'TradeMoney',
       'ExecType'],
      dtype='object')


In [3]:
for y in ['20200511']:    
    print('----------------------------------------------------------------')
    print('SH trade data:')
    
    SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
    SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum"]
    n1 = len(SH["SecurityID"].unique())
    n2 = len(SH1["SecurityID"].unique())
    print(n1)
    print(n2)
    if n1 > n2:
        SH = SH[SH["SecurityID"].isin(SH1["SecurityID"].unique())]
    if n2 > n1:
        SH1 = SH1[SH1["SecurityID"].isin(SH["SecurityID"].unique())]
    re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = re["sequenceNo"].count()
    n2 = re["TradeTime"].count()
    len1 = len(re)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        display("test is not complete:")
        display(re[np.isnan(re["TradeTime"])])
        print(len(re[np.isnan(re["TradeTime"])]))
        print(np.sort(re[np.isnan(re["TradeTime"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["TradeTime"])]["SecurityID"].unique()))
        print(re[np.isnan(re["TradeTime"])]["SecurityID"].unique())
    if (len1 == n2) & (n1 < len1):
        display("baseline is not complete:")
        display(re[np.isnan(re["sequenceNo"])])
        print(np.sort(re[np.isnan(re["sequenceNo"])]["TransactTime"].unique()))
        print(len(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique()))
        print(re[np.isnan(re["sequenceNo"])]["SecurityID"].unique())
        display(n2-n1)

----------------------------------------------------------------
SH trade data:
1598
1598
15913562
15912394
15913958
-----------------------------------------------


'test is not complete:'

,clockAtArrival,sequenceNo,exchId,TransactTime,ApplSeqNum,SecurityID,ExecType,TradeBSFlag,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,TradeTime,TradeChannel,SendingTime,TradeAmount
62438,1.589160e+15,2075298.0,1.0,92500050,10587,600735,F,N,52200,1500,78300000,122662,119581,NaN,NaN,NaN,NaN
62439,1.589160e+15,2075299.0,1.0,92500050,10588,600735,F,N,52200,300,15660000,149827,119581,NaN,NaN,NaN,NaN
62440,1.589160e+15,2075300.0,1.0,92500050,10589,600735,F,N,52200,600,31320000,149827,89433,NaN,NaN,NaN,NaN
62441,1.589160e+15,2075301.0,1.0,92500050,10590,600735,F,N,52200,100,5220000,149827,117212,NaN,NaN,NaN,NaN
62442,1.589160e+15,2075302.0,1.0,92500050,10591,600735,F,N,52200,300,15660000,149827,138459,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15849411,1.589180e+15,118358916.0,1.0,150000420,2842198,600735,F,N,52200,200,10440000,4639155,4595401,NaN,NaN,NaN,NaN
15849412,1.589180e+15,118358917.0,1.0,150000420,2842199,600735,F,N,52200,300,15660000,4647192,4595424,NaN,NaN,NaN,NaN
15849413,1.589180e+15,118358918.0,1.0,150000420,2842200,600735,F,N,52200,200,10440000,4647192,4595429,NaN,NaN,NaN,NaN
15849414,1.589180e+15,118358919.0,1.0,150000420,2842201,600735,F,N,52200,100,5220000,4647193,4595429,NaN,NaN,NaN,NaN


1564
[ 92500050  93000580  93000800  93001170  93001280  93001870  93003460
  93003570  93003660  93003990  93004450  93005690  93006070  93006700
  93006840  93009700  93011720  93011830  93012280  93013350  93013970
  93019790  93019910  93020050  93021110  93022990  93023330  93025960
  93026080  93029880  93030530  93031080  93032050  93035900  93038000
  93043610  93043640  93045730  93046890  93050370  93052980  93054580
  93055260  93055580  93055940  93056190  93056680  93056850  93057440
  93059060  93107010  93107040  93107140  93107200  93107510  93107540
  93108620  93109140  93109960  93110000  93110250  93113190  93117330
  93119400  93119430  93131440  93131500  93134340  93137450  93137540
  93137600  93142770  93144550  93145470  93147760  93149830  93149890
  93150240  93154980  93156280  93157880  93201490  93201760  93203980
  93207450  93207490  93207770  93213540  93213900  93215710  93216170
  93221830  93223190  93225490  93225700  93226140  93230930  93231610
 

In [15]:
SH1[SH1["SecurityID"] == 600735]["TradePrice"].unique()

array([31000., 30900., 31100., 31200., 31300., 30800.])

In [14]:
SH[SH["SecurityID"] == 600735]["TradePrice"].unique()

array([52200, 52300, 52400, 52500, 52600, 52700, 52800, 52100, 52000,
       51900], dtype=int64)